# GeoNet FDSN webservice with Obspy demo - GeoNet FDSN Clients

GeoNet operates two FDNS wave servers
- An archive server holds verified data starting 7 days after collection
- A near real-time servers holds unverified data for the last 8 days

There are a few different ways this could be done. 

## Import Modules and Define Clients

In [1]:
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

arc_client  = 'http://service.geonet.org.nz'
# or arc_client = "GEONET"
nrt_client = 'http://service-nrt.geonet.org.nz'

## Option 1: By trial

You will need to specify the correct wave server to get the data you want. If you try to get data from one server and that raises an exception (because the data you requested are not available) repeat the request with the other server.

In [2]:
t = UTCDateTime('2017-10-04')

#first try to get data from archive server
try:
    client = Client(arc_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('arc client successful')
#if this raises an exception, try the near real-time server
except:
    client = Client(nrt_client)
    st = client.get_waveforms('NZ', 'KRVZ', '10', 'EHZ', t, t + 300)
    print('nrt client successful')
print(st)

arc client successful
1 Trace(s) in Stream:
NZ.KRVZ.10.EHZ | 2017-10-03T23:59:59.283134Z - 2017-10-04T00:05:03.523134Z | 100.0 Hz, 30425 samples


## Option 2: Use the time range to select client
Use the time range to select a client, if the time range requires both clients then use Option 3. 

In [3]:
starttime = UTCDateTime.now()-518400 #6 days ago
endtime = starttime+300

days7 = UTCDateTime.now()-604800 #7 days ago
days8 = UTCDateTime.now()-691200 #8 days ago

if endtime < days7:
    client = Client(arc_client)
    print("Client is archive client")
elif starttime > days8:
    client = Client(nrt_client)
    print("Client is near real-time client")
else:
    print("Time range requires both clients")
    
st = client.get_waveforms('NZ', 'WEL', '10', 'HHZ', starttime, endtime)
print(st)

Client is near real-time client
1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2018-05-06T23:54:49.383133Z - 2018-05-06T23:59:51.013133Z | 100.0 Hz, 30164 samples


## Option 3: Request from both clients and merge
This is useful if the time window for the data request spans both time periods.

First, request data from both clients

In [4]:
#Define time period
t1 = UTCDateTime.now()-777600 #9 days ago
t2 = UTCDateTime.now()-518400 #6 days ago

#nrt client
strnt = None
starc = None
st = None
try:
    client = Client(nrt_client)
    stnrt = client.get_waveforms('NZ', 'WEL', '10', 'HHZ', t1, t2)
    print('nrt client successful')
    st = stnrt
except:
    print('nrt client not successful')

#arc client
try:
    client = Client(arc_client)
    starc = client.get_waveforms('NZ', 'WEL', '10', 'HHZ', t1, t2)
    print('arc client successful')
    st += starc
except:
    print('arc client not successful')
print(stnrt,starc)

nrt client successful
arc client successful
1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2018-05-04T23:54:20.743215Z - 2018-05-06T23:54:49.373215Z | 100.0 Hz, 17282864 samples 1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2018-05-03T23:54:47.223175Z - 2018-05-06T00:00:01.233175Z | 100.0 Hz, 17311402 samples


Now merge waveforms into a single stream object. 

In [5]:
st.merge(method=-1)
print(st)

1 Trace(s) in Stream:
NZ.WEL.10.HHZ | 2018-05-03T23:54:47.223175Z - 2018-05-06T23:54:49.373175Z | 100.0 Hz, 25920216 samples
